In [1]:
import pandas as pd
import numpy as np

In [2]:
DISPLAY_DF = True
DISPLAY_CALC = True
ROUNDING_POINT = 6

### Fill Problem Information (Manual) and Parse into DataFrames (Automated)

In [3]:
units_raw = {
    'unit'  :['U1'       ,'U2'       ,'U3'       ,'D1'                     ,'M1'       ],
    'role'  :['standard' ,'standard' ,'standard' ,'split'                  ,'standard' ],
    'inlet' :[['S1','S3'],['S2']     ,['S4']     ,['S6']                   ,['S7','S8']],
    'outlet':[['S2','S4'],['S5','S6'],['S8','S9'],['S3','S7']              ,['S10']    ],
    'ratio' :[{}         ,{}         ,{}         ,{'S3':0.50,'S7':0.50}    ,{}         ]
}

streams_raw = {
    'stream'    :['S1'                 ,'S2'       ,'S3'     ,'S4'         ,'S5'       ,'S6'     ,'S7'     ,'S8'       ,'S9'     ,'S10'                ],
    'species'   :[['A','B','C','D']    ,['A','B']  ,['A','B'],['B','C','D'],['A','B']  ,['A','B'],['A','B'],['B','C']  ,['C','D'],['A','B','C']        ],
    'mole_ratio':[{'A':0.264,'C':0.212},{'A':0.578},{}       ,{}           ,{'A':0.905},{}       ,{}       ,{'B':0.268},{}       ,{'A':0.108,'B':0.465}],
    'mole_flow' :[{}                   ,{}         ,{}       ,{}           ,{}         ,{}       ,{}       ,{}         ,{}       ,{}                   ]
}

# Create unit and stream DataFrames
units = pd.DataFrame(units_raw).set_index('unit')
streams = pd.DataFrame(streams_raw).set_index('stream')

In [4]:
# Parse upstream and downstream from inlet and outlet
def get_related_unit(S, col):
    #explodes the relevant column (inlet or outlet), filters for the rows there the new column matches S.name, then returns the index of the first match (or None if there were no matches)
    return next(iter(units.explode(col)[units.explode(col)[col] == S.name].index), None)

streams['upstream'] = streams.apply(get_related_unit, args=(['outlet']), axis = 1)
streams['downstream'] = streams.apply(get_related_unit, args=(['inlet']), axis = 1)

if DISPLAY_DF: display(units)
if DISPLAY_DF: display(streams)

#Code to get a DataFrame of the defined unit ratios for spliter units
#pd.json_normalize(units[units.role=="split"].ratio).set_index(units[units.role=="split"].index)

,role,inlet,outlet,ratio
unit,,,,
U1,standard,"[S1, S3]","[S2, S4]",{}
U2,standard,[S2],"[S5, S6]",{}
U3,standard,[S4],"[S8, S9]",{}
D1,split,[S6],"[S3, S7]","{'S3': 0.5, 'S7': 0.5}"
M1,standard,"[S7, S8]",[S10],{}


,species,mole_ratio,mole_flow,upstream,downstream
stream,,,,,
S1,"[A, B, C, D]","{'A': 0.264, 'C': 0.212}",{},None,U1
S2,"[A, B]",{'A': 0.578},{},U1,U2
S3,"[A, B]",{},{},D1,U1
S4,"[B, C, D]",{},{},U1,U3
S5,"[A, B]",{'A': 0.905},{},U2,None
S6,"[A, B]",{},{},U2,D1
S7,"[A, B]",{},{},D1,M1
S8,"[B, C]",{'B': 0.268},{},U3,M1
S9,"[C, D]",{},{},U3,None


In [5]:
# get a list of the component species
species = sorted(streams.explode('species').species.unique())

# parse 
n = pd.DataFrame(index=species)
n.index.name = 'n'
for S in streams.index:
    n[S] = [(np.nan if (s in streams.species[S]) else 0.00) for s in n.index]
n.loc['total'] = np.nan

x = pd.DataFrame(index=species)
x.index.name = 'x'
for S in streams.index:
    x[S] = [streams.mole_ratio[S].get(s, np.nan if (s in streams.species[S]) else 0.00) for s in x.index]

if DISPLAY_DF: display(n)
if DISPLAY_DF: display(x)

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
n,,,,,,,,,,
A,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN
B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
C,NaN,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN
D,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0
total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
x,,,,,,,,,,
A,0.264,0.578,NaN,0.0,0.905,NaN,NaN,0.000,0.0,0.108
B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.268,0.0,0.465
C,0.212,0.000,0.0,NaN,0.000,0.0,0.0,NaN,NaN,NaN
D,NaN,0.000,0.0,NaN,0.000,0.0,0.0,0.000,NaN,0.000


### Define Solving functions

In [6]:
def complete_ratios(df):
    changed = False
    for col in df.columns[df.isna().sum() == 1]:
        for i in df.index[df[col].isna()]:
            changed = True
            df.loc[i, col] = np.round(1 - sum([X for X in df.drop(i)[col]]),ROUNDING_POINT)
            if DISPLAY_CALC: print(f"{df.index.name}.{col}.{i} = 1 - {[f"{df.index.name}.{col}.{I}" for I in df.drop(i).index if df.loc[I, col] != 0.00]} = {df.loc[i,col]}")
    return changed
def complete_mole_ratios(): return complete_ratios(x)

def totals_from_components(df_col):
    sans_total = df_col.drop('total')
    if np.isnan(df_col.total) and not sans_total.isna().any():
        result = np.round(sum(sans_total),ROUNDING_POINT) 
        if DISPLAY_CALC: print(f"{df_col.index.name}.{df_col.name} = sum({[f"{df_col.index.name}.{df_col.name}.{I}" for I in sans_total.index if df_col.loc[I] != 0.00]}) = {result}")
        return result
    else:
        return df_col.total
def n_from_all_n_s(): n.loc['total'] = n.apply(totals_from_components, axis=0)

def component_from_total_and_ratio(a_df, r_df):
    for S in a_df.columns[~np.isnan(a_df.loc['total'])]:
        for i in r_df.index[~r_df[S].isna() & a_df.drop('total')[S].isna()]:
            a_df.loc[i, S] = np.round(a_df[S].total * r_df.loc[i, S],ROUNDING_POINT)
            if DISPLAY_CALC: print(f"{a_df.index.name}.{S}.{i} = {a_df.index.name}.{S} * {r_df.index.name}.{S}.{i} = {a_df.loc[i, S]}")
def n_s_from_n_and_x_s(): return component_from_total_and_ratio(n, x)

def ratio_from_component_and_total(a_df, r_df):
    for S in a_df.columns[~np.isnan(a_df.loc['total'])]:
        for i in r_df.index[r_df[S].isna() & ~a_df.drop('total')[S].isna()]:
            r_df.loc[i, S] = np.round(a_df.loc[i, S] / a_df[S].total,ROUNDING_POINT)
            if DISPLAY_CALC: print(f"{r_df.index.name}.{S}.{i} = {a_df.index.name}.{S}.{i} / {r_df.index.name}.{S} = {r_df.loc[i, S]}")
def x_s_from_n_s_and_n(): return ratio_from_component_and_total(n, x)

def total_from_component_and_ratio(a_df, r_df):
    for S in a_df.columns[np.isnan(a_df.loc['total'])]:
        for i in r_df.index[~a_df.drop('total')[S].isna() & r_df[S] > 0]:
            a_df[S].total = np.round(a_df.loc[i, S] / r_df.loc[i, S],ROUNDING_POINT)
            if DISPLAY_CALC: print(f"{a_df.index.name}.{S} = {a_df.index.name}.{S}.{i} / {r_df.index.name}.{S}.{i} = {a_df[S].total}")
            break
def n_from_n_s_and_x_s(): return total_from_component_and_ratio(n, x)

### Set basis, complete ratios, and calculate individual flow rates

In [7]:
n.S10.total = 1000 #mol/min
complete_mole_ratios()
n_s_from_n_and_x_s()

if DISPLAY_DF: display(n)
if DISPLAY_DF: display(x)

x.S2.B = 1 - ['x.S2.A'] = 0.422
x.S5.B = 1 - ['x.S5.A'] = 0.095
x.S8.C = 1 - ['x.S8.B'] = 0.732
x.S10.C = 1 - ['x.S10.A', 'x.S10.B'] = 0.427
n.S10.A = n.S10 * x.S10.A = 108.0
n.S10.B = n.S10 * x.S10.B = 465.0
n.S10.C = n.S10 * x.S10.C = 427.0


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
n,,,,,,,,,,
A,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,108.0
B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,465.0
C,NaN,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,427.0
D,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0
total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
x,,,,,,,,,,
A,0.264,0.578,NaN,0.0,0.905,NaN,NaN,0.000,0.0,0.108
B,NaN,0.422,NaN,NaN,0.095,NaN,NaN,0.268,0.0,0.465
C,0.212,0.000,0.0,NaN,0.000,0.0,0.0,0.732,NaN,0.427
D,NaN,0.000,0.0,NaN,0.000,0.0,0.0,0.000,NaN,0.000


### Solve for Mixing Point (S7, S8, S10)

In [8]:
n.S8.C = n.S10.C
n_from_n_s_and_x_s()
n_s_from_n_and_x_s()

if DISPLAY_DF: display(n)
if DISPLAY_DF: display(x)

n.S8 = n.S8.C / x.S8.C = 583.333333
n.S8.B = n.S8 * x.S8.B = 156.333333


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
n,,,,,,,,,,
A,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.000000,0.0,108.0
B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,156.333333,0.0,465.0
C,NaN,0.0,0.0,NaN,0.0,0.0,0.0,427.000000,NaN,427.0
D,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.000000,NaN,0.0
total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,583.333333,NaN,1000.0


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
x,,,,,,,,,,
A,0.264,0.578,NaN,0.0,0.905,NaN,NaN,0.000,0.0,0.108
B,NaN,0.422,NaN,NaN,0.095,NaN,NaN,0.268,0.0,0.465
C,0.212,0.000,0.0,NaN,0.000,0.0,0.0,0.732,NaN,0.427
D,NaN,0.000,0.0,NaN,0.000,0.0,0.0,0.000,NaN,0.000


In [9]:
n.S7.A = n.S10.A
n.S7.B = n.S10.B-n.S8.B
n_from_all_n_s()
x_s_from_n_s_and_n()

if DISPLAY_DF: display(n)
if DISPLAY_DF: display(x)

n.S7 = sum(['n.S7.A', 'n.S7.B']) = 416.666667
x.S7.A = n.S7.A / x.S7 = 0.2592
x.S7.B = n.S7.B / x.S7 = 0.7408


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
n,,,,,,,,,,
A,NaN,NaN,NaN,0.0,NaN,NaN,108.000000,0.000000,0.0,108.0
B,NaN,NaN,NaN,NaN,NaN,NaN,308.666667,156.333333,0.0,465.0
C,NaN,0.0,0.0,NaN,0.0,0.0,0.000000,427.000000,NaN,427.0
D,NaN,0.0,0.0,NaN,0.0,0.0,0.000000,0.000000,NaN,0.0
total,NaN,NaN,NaN,NaN,NaN,NaN,416.666667,583.333333,NaN,1000.0


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
x,,,,,,,,,,
A,0.264,0.578,NaN,0.0,0.905,NaN,0.2592,0.000,0.0,0.108
B,NaN,0.422,NaN,NaN,0.095,NaN,0.7408,0.268,0.0,0.465
C,0.212,0.000,0.0,NaN,0.000,0.0,0.0000,0.732,NaN,0.427
D,NaN,0.000,0.0,NaN,0.000,0.0,0.0000,0.000,NaN,0.000


### Solve For Splitter (S3, S6, S7)

In [10]:
x.S6.A = x.S7.A
x.S3.A = x.S6.A
n.S6.total = 2*n.S7.total
n.S3.total = n.S7.total
complete_mole_ratios()
n_s_from_n_and_x_s()

if DISPLAY_DF: display(n)
if DISPLAY_DF: display(x)

x.S3.B = 1 - ['x.S3.A'] = 0.7408
x.S6.B = 1 - ['x.S6.A'] = 0.7408
n.S3.A = n.S3 * x.S3.A = 108.0
n.S3.B = n.S3 * x.S3.B = 308.666667
n.S6.A = n.S6 * x.S6.A = 216.0
n.S6.B = n.S6 * x.S6.B = 617.333334


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
n,,,,,,,,,,
A,NaN,NaN,108.000000,0.0,NaN,216.000000,108.000000,0.000000,0.0,108.0
B,NaN,NaN,308.666667,NaN,NaN,617.333334,308.666667,156.333333,0.0,465.0
C,NaN,0.0,0.000000,NaN,0.0,0.000000,0.000000,427.000000,NaN,427.0
D,NaN,0.0,0.000000,NaN,0.0,0.000000,0.000000,0.000000,NaN,0.0
total,NaN,NaN,416.666667,NaN,NaN,833.333334,416.666667,583.333333,NaN,1000.0


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
x,,,,,,,,,,
A,0.264,0.578,0.2592,0.0,0.905,0.2592,0.2592,0.000,0.0,0.108
B,NaN,0.422,0.7408,NaN,0.095,0.7408,0.7408,0.268,0.0,0.465
C,0.212,0.000,0.0000,NaN,0.000,0.0000,0.0000,0.732,NaN,0.427
D,NaN,0.000,0.0000,NaN,0.000,0.0000,0.0000,0.000,NaN,0.000


### Solve for Unit II (S2, S5, S6)

In [11]:
lhs = x[['S2','S5']]['A':'B']
lhs.S5 = lhs.S5.apply(lambda X: -1*X)
rhs = n[['S6']]['A':'B']
n.loc['total',['S2','S5']] = np.linalg.solve(lhs,rhs).transpose()
display(lhs, rhs, n[['S2','S5']]['total':])
n_s_from_n_and_x_s()

if DISPLAY_DF: display(n)
if DISPLAY_DF: display(x)

,S2,S5
x,,
A,0.578,-0.905
B,0.422,-0.095


,S6
n,
A,216.000000
B,617.333334


,S2,S5
n,,
total,1645.769625,812.436291


n.S2.A = n.S2 * x.S2.A = 951.254843
n.S2.B = n.S2 * x.S2.B = 694.514782
n.S5.A = n.S5 * x.S5.A = 735.254843
n.S5.B = n.S5 * x.S5.B = 77.181448


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
n,,,,,,,,,,
A,NaN,951.254843,108.000000,0.0,735.254843,216.000000,108.000000,0.000000,0.0,108.0
B,NaN,694.514782,308.666667,NaN,77.181448,617.333334,308.666667,156.333333,0.0,465.0
C,NaN,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,427.000000,NaN,427.0
D,NaN,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.0
total,NaN,1645.769625,416.666667,NaN,812.436291,833.333334,416.666667,583.333333,NaN,1000.0


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
x,,,,,,,,,,
A,0.264,0.578,0.2592,0.0,0.905,0.2592,0.2592,0.000,0.0,0.108
B,NaN,0.422,0.7408,NaN,0.095,0.7408,0.7408,0.268,0.0,0.465
C,0.212,0.000,0.0000,NaN,0.000,0.0000,0.0000,0.732,NaN,0.427
D,NaN,0.000,0.0000,NaN,0.000,0.0000,0.0000,0.000,NaN,0.000


### Solve for Unit I (S1, S2, S3, S4)

In [12]:
n.S1.A = n.S5.A + n.S10.A
n.S1.B = n.S5.B + n.S10.B
n_from_n_s_and_x_s()
x_s_from_n_s_and_n()
complete_mole_ratios()
n_s_from_n_and_x_s()

if DISPLAY_DF: display(n)
if DISPLAY_DF: display(x)

n.S1 = n.S1.A / x.S1.A = 3194.147133
x.S1.B = n.S1.B / x.S1 = 0.169742
x.S1.D = 1 - ['x.S1.A', 'x.S1.B', 'x.S1.C'] = 0.354258
n.S1.C = n.S1 * x.S1.C = 677.159192
n.S1.D = n.S1 * x.S1.D = 1131.552175


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
n,,,,,,,,,,
A,843.254843,951.254843,108.000000,0.0,735.254843,216.000000,108.000000,0.000000,0.0,108.0
B,542.181448,694.514782,308.666667,NaN,77.181448,617.333334,308.666667,156.333333,0.0,465.0
C,677.159192,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,427.000000,NaN,427.0
D,1131.552175,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.0
total,3194.147133,1645.769625,416.666667,NaN,812.436291,833.333334,416.666667,583.333333,NaN,1000.0


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
x,,,,,,,,,,
A,0.264000,0.578,0.2592,0.0,0.905,0.2592,0.2592,0.000,0.0,0.108
B,0.169742,0.422,0.7408,NaN,0.095,0.7408,0.7408,0.268,0.0,0.465
C,0.212000,0.000,0.0000,NaN,0.000,0.0000,0.0000,0.732,NaN,0.427
D,0.354258,0.000,0.0000,NaN,0.000,0.0000,0.0000,0.000,NaN,0.000


In [13]:
n.S4.B = n.S1.B + n.S3.B - n.S2.B
n.S4.C = n.S1.C
n.S4.D = n.S1.D
n_from_all_n_s()
x_s_from_n_s_and_n()

if DISPLAY_DF: display(n)
if DISPLAY_DF: display(x)

n.S4 = sum(['n.S4.B', 'n.S4.C', 'n.S4.D']) = 1965.0447
x.S4.B = n.S4.B / x.S4 = 0.079557
x.S4.C = n.S4.C / x.S4 = 0.344602
x.S4.D = n.S4.D / x.S4 = 0.57584


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
n,,,,,,,,,,
A,843.254843,951.254843,108.000000,0.000000,735.254843,216.000000,108.000000,0.000000,0.0,108.0
B,542.181448,694.514782,308.666667,156.333333,77.181448,617.333334,308.666667,156.333333,0.0,465.0
C,677.159192,0.000000,0.000000,677.159192,0.000000,0.000000,0.000000,427.000000,NaN,427.0
D,1131.552175,0.000000,0.000000,1131.552175,0.000000,0.000000,0.000000,0.000000,NaN,0.0
total,3194.147133,1645.769625,416.666667,1965.044700,812.436291,833.333334,416.666667,583.333333,NaN,1000.0


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
x,,,,,,,,,,
A,0.264000,0.578,0.2592,0.000000,0.905,0.2592,0.2592,0.000,0.0,0.108
B,0.169742,0.422,0.7408,0.079557,0.095,0.7408,0.7408,0.268,0.0,0.465
C,0.212000,0.000,0.0000,0.344602,0.000,0.0000,0.0000,0.732,NaN,0.427
D,0.354258,0.000,0.0000,0.575840,0.000,0.0000,0.0000,0.000,NaN,0.000


### Solve for Unit III (S4, S8, S9)

In [14]:
n.S9.C = n.S4.C - n.S8.C
n.S9.D = n.S4.D
n_from_all_n_s()
x_s_from_n_s_and_n()

if DISPLAY_DF: display(n)
if DISPLAY_DF: display(x)

n.S9 = sum(['n.S9.C', 'n.S9.D']) = 1381.711367
x.S9.C = n.S9.C / x.S9 = 0.18105
x.S9.D = n.S9.D / x.S9 = 0.81895


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
n,,,,,,,,,,
A,843.254843,951.254843,108.000000,0.000000,735.254843,216.000000,108.000000,0.000000,0.000000,108.0
B,542.181448,694.514782,308.666667,156.333333,77.181448,617.333334,308.666667,156.333333,0.000000,465.0
C,677.159192,0.000000,0.000000,677.159192,0.000000,0.000000,0.000000,427.000000,250.159192,427.0
D,1131.552175,0.000000,0.000000,1131.552175,0.000000,0.000000,0.000000,0.000000,1131.552175,0.0
total,3194.147133,1645.769625,416.666667,1965.044700,812.436291,833.333334,416.666667,583.333333,1381.711367,1000.0


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
x,,,,,,,,,,
A,0.264000,0.578,0.2592,0.000000,0.905,0.2592,0.2592,0.000,0.00000,0.108
B,0.169742,0.422,0.7408,0.079557,0.095,0.7408,0.7408,0.268,0.00000,0.465
C,0.212000,0.000,0.0000,0.344602,0.000,0.0000,0.0000,0.732,0.18105,0.427
D,0.354258,0.000,0.0000,0.575840,0.000,0.0000,0.0000,0.000,0.81895,0.000


## Calculate Solution

In [15]:
print(f"n_2/n_4 = {n.S2.total/n.S4.total:2.1%}")

display(n)
display(x)

n_2/n_4 = 83.8%


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
n,,,,,,,,,,
A,843.254843,951.254843,108.000000,0.000000,735.254843,216.000000,108.000000,0.000000,0.000000,108.0
B,542.181448,694.514782,308.666667,156.333333,77.181448,617.333334,308.666667,156.333333,0.000000,465.0
C,677.159192,0.000000,0.000000,677.159192,0.000000,0.000000,0.000000,427.000000,250.159192,427.0
D,1131.552175,0.000000,0.000000,1131.552175,0.000000,0.000000,0.000000,0.000000,1131.552175,0.0
total,3194.147133,1645.769625,416.666667,1965.044700,812.436291,833.333334,416.666667,583.333333,1381.711367,1000.0


,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10
x,,,,,,,,,,
A,0.264000,0.578,0.2592,0.000000,0.905,0.2592,0.2592,0.000,0.00000,0.108
B,0.169742,0.422,0.7408,0.079557,0.095,0.7408,0.7408,0.268,0.00000,0.465
C,0.212000,0.000,0.0000,0.344602,0.000,0.0000,0.0000,0.732,0.18105,0.427
D,0.354258,0.000,0.0000,0.575840,0.000,0.0000,0.0000,0.000,0.81895,0.000
